In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("./../dataprocess/Emergency_Response_Incidents.csv", error_bad_lines=False, index_col=False) #9018
EMOriginalRT = df
#EMOriginalRT = df[(df[clasify].str.startswith("Fire") == True)|(df[clasify].str.startswith("Utility") == True)|(df[clasify].str.startswith("Law Enforcement") == True)|(df[clasify].str.startswith("Structural") == True)]

b'Skipping line 605: expected 7 fields, saw 9\nSkipping line 1740: expected 7 fields, saw 8\nSkipping line 2534: expected 7 fields, saw 9\nSkipping line 2854: expected 7 fields, saw 8\nSkipping line 2976: expected 7 fields, saw 9\nSkipping line 3752: expected 7 fields, saw 8\n'


In [3]:
# CREATE RESPONSE TIME COLUMN

EMOriginalRT["Creation Date"]=pd.to_datetime(EMOriginalRT["Creation Date"])
EMOriginalRT = EMOriginalRT.sort_values(by=["Creation Date"])
EMOriginalRT["Response Time"] = (pd.to_datetime(EMOriginalRT["Closed Date"]) - pd.to_datetime(EMOriginalRT["Creation Date"])).dt.total_seconds()
EMOriginalRT["minute"] = 60
EMOriginalRT["Response Time"] = EMOriginalRT["Response Time"].div(EMOriginalRT["minute"])
EMOriginalRT = EMOriginalRT.drop(["minute"], axis=1)

In [4]:
EMOriginalRT["Response Time"].quantile(0.9)

4127.716666666667

In [5]:
# CREATE GROUP TYPE WITH FIRST 3 LETTERS OF INCIDENT TYPE
EMOriginalRT['Group Type'] = EMOriginalRT['Incident Type'].str[:3]

In [6]:
#df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})
#EMOriginalRT = EMOriginalRT.dropna(subset=["Closed Date"])


In [7]:
print(EMOriginalRT["Creation Date"].min())
print(EMOriginalRT["Creation Date"].max())

2011-05-04 06:03:58
2019-12-17 20:19:18


In [8]:
# MARK OUTLIERS AND DO NOT COUNT THEM

In [40]:
listGroups = EMOriginalRT.groupby('Group Type').groups.keys()#.agg()#['Group Type']

In [38]:
EMOriginalRT.groups.keys()

In [10]:
listGroups = EMOriginalRT.groupby('Group Type')['Response Time'].quantile(0.9)#.agg(['mean','quantile(0.9)'])#..quantile(.90)

In [11]:
EMOriginalRT["NoCloseDate"]=0
EMOriginalRT["Outlier"]=0


In [41]:
## SORT BY CREATION DATE

#EMOriginalRT = EMOriginalRT.sort_values(by=["Creation Date"])
listGroups.index

AttributeError: 'dict_keys' object has no attribute 'index'

In [42]:
listGroups.describe()#['Group Type']

AttributeError: 'dict_keys' object has no attribute 'describe'

In [44]:
listDfPerGroup = []
for groupStr in listGroups:
    print(groupStr)
    groupDF = pd.DataFrame(EMOriginalRT[(EMOriginalRT['Group Type'] == groupStr)])
    listDfPerGroup.append(groupDF)


Adm
Avi
Fir
Haz
Law
Mar
Med
Oth
Pow
Res
Str
Tra
Uti
Wea


In [15]:
len(listDfPerGroup)#14

14

In [16]:
'''
listDateRange = [('2011-05','2011-11'),('2011-11','2012-05'),('2012-05','2012-11'),('2012-11','2013-05'),
                 ('2013-05','2013-11'),('2013-11','2014-05'),('2014-05','2014-11'),('2014-11','2015-05'),
                 ('2015-05','2015-11'),('2015-11','2016-05'),('2016-05','2016-11'),('2016-11','2017-05'),
                 ('2017-05','2017-11'),('2017-11','2018-05'),('2018-05','2018-11'),('2018-11','2019-05'),
                 ('2019-05','2020-01')]
'''
listDateRange = [('2011-05','2012-05'),('2012-05','2013-05'),('2013-05','2014-05'),('2014-05','2015-05'),
                 ('2015-05','2016-05'),('2016-05','2017-05'),('2017-05','2018-05'),('2018-05','2020-01')]
#'''

listSplits = ['train','validation','test']
dictSplit = {'train':0.5,'validation':0.25,'test':0.25}

In [18]:
listPermu

[('train', 'validation', 'test'),
 ('train', 'test', 'validation'),
 ('validation', 'train', 'test'),
 ('validation', 'test', 'train'),
 ('test', 'train', 'validation'),
 ('test', 'validation', 'train')]

In [19]:
def create_inout_sequences(input_data, xw, yw):
    '''
    Input: Sequence or dataFrame of one column ('Response Time')
    Output: lists of X and list of Y
    '''
    in_seq = []
    out_seq = []
    L = len(input_data)
    for i in range(L-xw-yw):
        train_seq = input_data[i:i+xw]
        train_label = input_data[i+xw:i+xw+yw]
        in_seq.append(train_seq )
        out_seq.append(train_label)
    return in_seq, out_seq

In [20]:
def split(all_data, listSplit):
    '''
    Input: dataframe, could be more than one column
    Output: lists of X and list of Y
    '''
    
    dictDFrames = {}
    #the subRange in the period, if access with loc has positions from 0 to shape[0]
    prevIdx = 0 #all_data.index[0]
    originalSize = all_data.shape[0]
    for setSplit in listSplit:
        endIdx = int(dictSplit[setSplit]*originalSize)
        
        dictDFrames[setSplit] = all_data.iloc[prevIdx: prevIdx + endIdx]
        
        #print(prevIdx,prevIdx + endIdx, dictDFrames[setSplit].shape[0])
        
        prevIdx = prevIdx + endIdx
        

    return dictDFrames

In [21]:
x_length = 15
y_length = 5
per_split = 0.5

In [46]:
def minMaxData(train_data, valid_data, test_data):
    #Preprocessing
    scaler = MinMaxScaler()#feature_range=(-1, 1)
    scaler.fit(train_data)
    train_normalized = scaler.transform(train_data)#.reshape(-1, 1)
    valid_normalized = scaler.transform(valid_data.reshape(-1,1))
    test_normalized = scaler.transform(test_data.reshape(-1, 1))

    return train_normalized, valid_normalized, test_normalized, scaler

In [47]:
listDfPerGroup[0].loc[(listDfPerGroup[0]['Response Time']>4000) | (listDfPerGroup[0]["Response Time"].isna()==True)]

,Incident Type,Location,Borough,Creation Date,Closed Date,Latitude,Longitude,Response Time,Group Type,NoCloseDate,Outlier
1079,Administration+AC0-Evacuation Drill,NYS Courts+AC0- Queens and Staten Island,Citywide,2011-05-10 08:17:30,NaN,NaN,NaN,NaN,Adm,0,0
1812,Administration+AC0-Planned Event,Black Eyed Peas Concert,Manhattan,2011-09-21 12:46:13,10/01/2011 03:02:00 AM,40.785294,+AC0-73.9693347761,13815.783333,Adm,0,0
6521,Administration+AC0-Planned Event,NaN,Citywide,2012-06-19 19:59:15,07/05/2012 12:28:34 AM,40.744435,+AC0-73.9665737457,21869.316667,Adm,0,0
6657,Administration+AC0-Planned Event,NaN,Queens,2012-06-19 20:00:17,06/25/2012 04:25:08 AM,NaN,NaN,7704.850000,Adm,0,0
2953,Administration+AC0-Evacuation Drill,45 Monroe Place,Brooklyn,2012-09-12 11:33:17,10/17/2012 03:22:51 PM,40.695821,+AC0-73.9928019729,50629.566667,Adm,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1056,Administration+AC0-Planned Event,NaN,Brooklyn,2018-10-07 18:46:48,NaN,NaN,NaN,NaN,Adm,0,0
3427,Administration+AC0-Drill,NaN,Brooklyn,2018-10-30 07:02:42,NaN,NaN,NaN,NaN,Adm,0,0
3086,Administration+AC0-Planned Event,NaN,Brooklyn,2018-11-20 16:06:29,NaN,NaN,NaN,NaN,Adm,0,0
6080,Administration+AC0-Planned Event,NaN,Brooklyn,2018-11-20 16:06:29,NaN,NaN,NaN,NaN,Adm,0,0


In [48]:
list(listDfPerGroup[0]['Response Time'])

[nan,
 400.48333333333335,
 13815.783333333333,
 165.48333333333332,
 2723.6833333333334,
 2576.483333333333,
 127.38333333333335,
 21869.316666666666,
 7704.85,
 2991.6833333333334,
 3201.05,
 50629.566666666666,
 53196.55,
 280.95,
 5805.316666666667,
 4579.483333333334,
 97.28333333333333,
 1300.6166666666666,
 472.8666666666667,
 736.0833333333334,
 1416.9333333333334,
 303.6,
 25562.383333333335,
 33.75000000000001,
 121072.31666666667,
 46125.23333333333,
 7495.05,
 2927.35,
 155855.23333333334,
 9464.966666666667,
 2408.9666666666667,
 2007.3000000000002,
 20.683333333333334,
 55.68333333333333,
 48.9,
 28251.683333333334,
 2446.0333333333333,
 2429.3333333333335,
 nan,
 85114.45,
 94707.28333333334,
 134843.4166666667,
 9093.733333333334,
 103.11666666666666,
 54150.3,
 5962.683333333333,
 4543.35,
 14.1,
 118702.51666666666,
 13880.416666666666,
 3047.483333333333,
 860.8666666666667,
 2748.25,
 4494.783333333334,
 130361.58333333334,
 2841.7166666666667,
 457.8333333333333,
 

In [60]:
import h5py
import random
from sklearn.preprocessing import MinMaxScaler


############# PENDING
### Save indexes of what was training, val and split form the origina

for dfGroup in listDfPerGroup:
    # We have to append instances of different periods as different dataframes because later
    # they would be transform to sequences
    
    
    dfGroup = dfGroup.reset_index()    
    groupType = dfGroup['Group Type'].loc[0]
    
    groupType+='_'+listDateRange[0][0]+'_'+listDateRange[0][1]+'_'+str(x_length)+'_'+str(y_length)
    print(groupType)
    hf = h5py.File('./../Data/MixedPeriods/'+groupType+'.h5', 'w')
    
    
    trainList = []
    valList = []
    testList = []
    
    countPermuSplits = 0
    
    # Get the split along all periods
    for iniDate, endDate in listDateRange:
        dfInRange = dfGroup.loc[(dfGroup["Creation Date"]>=iniDate)  & (dfGroup["Creation Date"]<endDate)]
        #print("Size group: ",dfInRange.shape[0])
        #receive a dctionary better
        dictDFrames = split(dfInRange, listPermu[countPermuSplits%3])
        
        tra = dictDFrames['train']
        val = dictDFrames['validation']
        tst = dictDFrames['test']
        
        #Appending in one list all the pieces from different periods
        #print(dfInRange.shape[0],tra.shape[0], tra.index[:3], val.shape[0], tst.shape[0])
        trainList.append(tra)
        valList.append(val)
        testList.append(tst)
        
        countPermuSplits +=1
        
    #Get quantiles in training
    #print(len(trainList))
    oneSeriesTrain = pd.concat(trainList)
    
    #In case there are no train instances at all:
    if oneSeriesTrain.shape[0]==0:
        hf.close()
        continue
    #print(oneSeriesTrain.shape[0], oneSeriesTrain.index)
    oneSeriesVal = pd.concat(valList)
    oneSeriesTest = pd.concat(testList)
    
    q50Train = oneSeriesTrain.quantile(0.5)['Response Time']
    q90Train = oneSeriesTrain.quantile(0.9)['Response Time']

    q50Valid = oneSeriesVal.quantile(0.5)['Response Time']
    q90Valid = oneSeriesVal.quantile(0.9)['Response Time']
    
    q50Test = oneSeriesTest.quantile(0.5)['Response Time']
    q90Test = oneSeriesTest.quantile(0.9)['Response Time']
    
    # Replace no close-date and outliers
    print(q90Train)
    outliers = dfGroup.loc[(dfGroup['Response Time']>q90Train) | (dfGroup["Response Time"].isna()==True)]
    validPoints = dfGroup.loc[(dfGroup['Response Time']<=q90Train)]
    #print(dfGroup.shape[0], outliers.shape[0], validPoints.shape[0])
    #print(outliers)
    
    # Choices instead of sample to include repetition in cases where the sampled list is smaller
    # than the values to complete
    newValues = random.choices(list(validPoints['Response Time']), k=outliers.shape[0])
    #print(newValues[:3], len(newValues) )
    
    #print(dfGroup['Response Time'].iloc[outliers.index])
    dfGroup['Response Time'].iloc[outliers.index] = newValues
    #print(dfGroup['Response Time'].iloc[outliers.index])
    #print(dfGroup.loc[dfGroup["Response Time"].isna()==True].shape[0])#0 correct
    
    # Normalize
    #print(dfGroup['Response Time'].iloc[oneSeriesTrain.index].values.astype(float)[:3],
    #      dfGroup['Response Time'].iloc[oneSeriesTrain.index].shape,
    #      dfGroup['Response Time'].iloc[oneSeriesTest.index].shape 
    #     )
    #.values.astype(float)

    
    #None to make shape (x,1) needed for fit that expected 2D
    train, validation, test, scaler = minMaxData(dfGroup['Response Time'].iloc[oneSeriesTrain.index][:,None],
                                                 dfGroup['Response Time'].iloc[oneSeriesVal.index][:,None], 
                                                 dfGroup['Response Time'].iloc[oneSeriesTest.index][:,None])
    
    newDFGroup = pd.DataFrame()
    newDFGroup['Response Time'] = dfGroup['Response Time']
    print(newDFGroup['Response Time'].iloc[oneSeriesTrain.index])
    newDFGroup['Response Time'].iloc[oneSeriesTrain.index] = np.squeeze(train)
    newDFGroup['Response Time'].iloc[oneSeriesVal.index] = np.squeeze(validation)
    newDFGroup['Response Time'].iloc[oneSeriesTest.index] = np.squeeze(test)
    
    print(newDFGroup['Response Time'].iloc[oneSeriesTrain.index])
    # Transform to sequences
    newDFTrainX = []
    newDFTrainY = []
    for batch in trainList:
        batchTransformedX, batchTransformedY  = create_inout_sequences(newDFGroup['Response Time'].iloc[batch.index] , x_length, y_length)
        #print("Batch added Train ", len(batchTransformedX), len(batchTransformedY))
        
        #In order for later be able to concatenate 0,0 with m,n
        if len(batchTransformedX)!=0:
            newDFTrainX.append(batchTransformedX)
            newDFTrainY.append(batchTransformedY)
    #print(len(newDFTrainX))
    
    newDFValX = []
    newDFValY = []
    for batch in valList:
        batchTransformedX, batchTransformedY = create_inout_sequences(newDFGroup['Response Time'].iloc[batch.index] , x_length, y_length)
        #print("Batch added Val ", len(batchTransformedX), len(batchTransformedY))
        if len(batchTransformedX)!=0:
            newDFValX.append(batchTransformedX)
            newDFValY.append(batchTransformedY)
    #print(len(newDFVal))
    
    newDFTestX = []
    newDFTestY = []
    for batch in testList:
        batchTransformedX, batchTransformedY = create_inout_sequences(newDFGroup['Response Time'].iloc[batch.index] , x_length, y_length)
        #print("Batch added Test ", len(batchTransformedX), len(batchTransformedY))
        if len(batchTransformedX)!=0:
            newDFTestX.append(batchTransformedX)
            newDFTestY.append(batchTransformedY)
            
    if len(newDFTrainX)==0 or len(newDFValX)==0 or len(newDFTestX)==0:
        hf.close()
        continue
    newDFTrainX = np.concatenate(newDFTrainX, axis=0)
    newDFTrainY = np.concatenate(newDFTrainY, axis=0)

    newDFValX = np.concatenate(newDFValX, axis=0)
    newDFValY = np.concatenate(newDFValY, axis=0)
    newDFTestX = np.concatenate(newDFTestX, axis=0)
    newDFTestY = np.concatenate(newDFTestY, axis=0)
    
    #dfGroup['Group Type'].loc[0],
    print( newDFTrainX.shape, newDFValX.shape, newDFTestX.shape )
    
    '''
    dataSetFinal = {}
    dataSetFinal['trainX'] = newDFTrainX
    dataSetFinal['trainY'] = newDFTrainY
    dataSetFinal['validX'] = newDFValX
    dataSetFinal['validY'] = newDFValY    
    dataSetFinal['testX'] = newDFTestX
    dataSetFinal['testY'] = newDFTestY
    '''
    
    #Creating H5FD file
    hf.create_dataset('trainX', data=newDFTrainX)
    hf.create_dataset('trainY', data=newDFTrainY)
    hf.create_dataset('validX', data=newDFValX)
    hf.create_dataset('validY', data=newDFValY)
    hf.create_dataset('testX', data=newDFTestX)
    hf.create_dataset('testY', data=newDFTestY)
    hf.create_dataset('quantiles', data = [q50Train, q90Train, q50Valid, q90Valid, q50Test, q90Test])
    
    hf.close()
    
    plt.hist(newDFTrainX.reshape(-1,1))
    plt.savefig("{}_trainX".format(groupType), bbox_inches='tight')
    plt.clf()
    
    plt.hist(newDFValX.reshape(-1,1))
    plt.savefig("{}_valX".format(groupType), bbox_inches='tight')
    plt.clf()

    plt.hist(newDFTestX.reshape(-1,1))
    plt.savefig("{}_testX".format(groupType), bbox_inches='tight')
    plt.clf()

    plt.hist(newDFTrainY.reshape(-1,1))
    plt.savefig("{}_trainY".format(groupType), bbox_inches='tight')
    plt.clf()
    
    plt.hist(newDFValY.reshape(-1,1))
    plt.savefig("{}_valY".format(groupType), bbox_inches='tight')
    plt.clf()

    plt.hist(newDFTestY.reshape(-1,1))
    plt.savefig("{}_testY".format(groupType), bbox_inches='tight')
    plt.clf()
    


Adm_2011-05_2012-05_15_5
23715.85
0        457.833333
1        400.483333
2      13815.783333
6        127.383333
7      21869.316667
           ...     
140    18809.066667
141     1778.350000
142     1533.650000
143     1451.200000
144     1416.933333
Name: Response Time, Length: 77, dtype: float64
0      0.020786
1      0.018164
2      0.631687
6      0.005674
7      1.000000
         ...   
140    0.860045
141    0.081178
142    0.069987
143    0.066216
144    0.064649
Name: Response Time, Length: 77, dtype: float64
Avi_2011-05_2012-05_15_5
2548.616666666667
0       32.483333
2      161.883333
6     1398.783333
7      621.933333
8       32.483333
12     172.300000
13      31.600000
17     137.900000
20     145.083333
23     300.283333
24     108.483333
27     108.483333
28     300.283333
29    2548.616667
30      11.483333
31      32.483333
Name: Response Time, dtype: float64
0     0.008277
2     0.059280
6     0.546798
7     0.240606
8     0.008277
12    0.063385
13    0.007929
17

/home/gissella/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/gissella/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/gissella/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

5738.30333333333
0       3471.266667
1       1145.516667
2          4.050000
3       2097.416667
4        112.116667
           ...     
2600     577.400000
2601     107.716667
2602     144.216667
2603     175.916667
2604      31.283333
Name: Response Time, Length: 1527, dtype: float64
0       0.605866
1       0.199935
2       0.000707
3       0.366078
4       0.019569
          ...   
2600    0.100778
2601    0.018801
2602    0.025171
2603    0.030704
2604    0.005460
Name: Response Time, Length: 1527, dtype: float64
(1367, 15) (601, 15) (601, 15)
Haz_2011-05_2012-05_15_5
6485.216666666712
0        67.200000
1         6.383333
2      2801.033333
3       133.433333
4        50.550000
          ...     
485      61.433333
486    3894.066667
487    2838.783333
488    3948.300000
489     103.750000
Name: Response Time, Length: 267, dtype: float64
0      0.013338
1      0.000155
2      0.605945
3      0.027696
4      0.009729
         ...   
485    0.012088
486    0.842880
487    0.614128


/home/gissella/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Law_2011-05_2012-05_15_5
2138.2833333333333
0       42.566667
1       67.116667
2      203.983333
3       79.516667
4       74.450000
          ...    
942     27.666667
943     69.300000
944     42.750000
945     42.366667
946     52.333333
Name: Response Time, Length: 517, dtype: float64
0      0.021579
1      0.034024
2      0.103407
3      0.040310
4      0.037742
         ...   
942    0.014025
943    0.035131
944    0.021672
945    0.021477
946    0.026530
Name: Response Time, Length: 517, dtype: float64


...

Uti_2011-05_2012-05_15_5
2999.275
0        787.000000
1       1688.016667
2        235.850000
3        120.800000
4        371.666667
           ...     
1654     150.116667
1655      72.100000
1656     145.350000
1657     410.633333
1658     172.100000
Name: Response Time, Length: 940, dtype: float64
0       0.265384
1       0.569215
2       0.079531
3       0.040735
4       0.125329
          ...   
1654    0.050621
1655    0.024313
1656    0.049013
1657    0.138469
1658    0.058034
Name: Response Time, Length: 940, dtype: float64


/home/gissella/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


(780, 15) (308, 15) (308, 15)
Wea_2011-05_2012-05_15_5
1923.3116666666685
0      742.683333
2     1136.216667
3      536.383333
4      129.466667
5       60.716667
12     703.933333
13     235.366667
14      24.000000
17     235.400000
18     117.933333
22      17.466667
23      25.466667
24     322.566667
25     522.083333
26      46.833333
33     145.366667
34     289.233333
36     338.400000
37     162.033333
38     142.283333
39      73.833333
44      75.283333
45     162.033333
46     120.366667
Name: Response Time, dtype: float64
0     0.648238
2     1.000000
3     0.463836
4     0.100112
5     0.038659
12    0.613601
13    0.194771
14    0.005840
17    0.194801
18    0.089803
22    0.000000
23    0.007151
24    0.272715
25    0.451054
26    0.026250
33    0.114324
34    0.242920
36    0.286868
37    0.129222
38    0.111568
39    0.050384
44    0.051680
45    0.129222
46    0.091978
Name: Response Time, dtype: float64


/home/gissella/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<Figure size 432x288 with 0 Axes>

In [ ]:
#'>=' not supported between instances of 'str' and 'int'
#Usually because
#ValueError: Can only compare identically-labeled Series objects
#TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

'''

## 17 periods of semesters
Adm 53382.96
Avi 1398.78
Fir 5223.51 (1184, 15) (430, 15) (430, 15)
Haz 11994.24
Law 691.53 (176, 15) (6, 15) (6, 15)
Mar 3701.97
Med 5575.04
Oth 900.99
Pow
Res 638.43
Str 5345.65 (409, 15) (60, 15) (60, 15)
Tra 1413.80
Uti 2622.66 (599, 15) (144, 15) (144, 15)
Wea 2622.70

## 8 periods of years
Adm 23715.85
Avi 2548.61
Fir 5738.30 (1367, 15) (601, 15) (601, 15)
Haz 6485.21 (111, 15) (6, 15) (6, 15)
Law 2138.28 (357, 15) (96, 15) (96, 15)
Mar 19170.41
Med 5633.16
Oth 827.71
Pow
Res
466.74
Str 6862.82 (592, 15) (213, 15) (213, 15)
Tra 2541.17 (51, 15) (7, 15) (7, 15)
Uti 2999.275 (780, 15) (308, 15) (308, 15)
Wea 1923.31

'''